# Lab-1 

Name: Ruozhang Yao


## 1.Executive summary
The capacity rationing policy is recommended because it’s more flexible priding strategies in terms of demand and current capacity.

• Optimal 245.0 for the constant pricing policy and the associated expected profit 2633.10.

• Optimal 240.0 and 300.0 for the dynamic pricing policy and the associated expected profit 3506.80

• Optimal 230.0, 335.0 and 28.0 for the capacity rationing policy and the associated expected profit 3559.75

It raised about ``35.2%`` gain in expected profit compared to the current constant pricing strategy. 

In [5]:
from scipy.stats import norm
from scipy.stats import uniform
import pandas as pd
import numpy as np
from scipy.stats import uniform 

## 2.Generating Customers

In [6]:
def generateCustomers():
    leisureSize  = norm(200,60)
    leisurePay = norm(200,80)
    leisurePurchase  = uniform(loc =0, scale = 180)
    
    businessSize  = norm(60,20)
    businessPay = norm(350,120)
    businessPurchase  = uniform(loc =0, scale = 14)
    
    leisureSample = max(int(leisureSize.rvs()),0)
    businessSample = max(int(businessSize.rvs()),0)
    
    valuation = []
    arrival_time = []
    
    for i in range(leisureSample):
        arrival_time.append(leisurePurchase.rvs())
        valuation.append(int(leisurePay.rvs()))
    frame1 = pd.DataFrame({'arrival_time':arrival_time, 'valuation':valuation})
    
    valuation = []
    arrival_time = []
    
    for i in range(businessSample):
        arrival_time.append(businessPurchase.rvs())
        valuation.append(int(businessPay.rvs()))
    frame2 = pd.DataFrame({'arrival_time':arrival_time, 'valuation':valuation})
    
    customers = pd.concat([frame1, frame2])
    
    customers = customers.sort_values(by = ['arrival_time'],ascending = False)
    customers = customers.reset_index()
    customers = customers.drop(columns= 'index')
    return customers


In [7]:
import numpy as np
np.random.seed(0)
generateCustomers().plot(x='arrival_time',y='valuation',kind='scatter')

In [8]:
generateCustomers()

,arrival_time,valuation
0,179.638920,132
1,179.246753,193
2,177.820050,121
3,177.319562,208
4,175.517042,158
5,173.231113,222
6,170.737254,304
7,170.607940,128
8,169.893154,98
9,169.048964,202


## 3.  Simulating Pricing Policies

In [9]:
def simulateConstant(customers, seats, p):   
    countTicket = seats
    revenue = 0
    for x in customers['valuation']:
        if x >= p:
            countTicket = countTicket - 1
            revenue = revenue + p
        if countTicket == 0:
            break
    
    return revenue

In [10]:
import pandas as pd
data=[[170,50],[150,40],[13,95],[12,100]]
customers=pd.DataFrame(data,columns=['arrival_time','valuation'])
print('Constant policy with 2 seats p=50 \t:',simulateConstant(customers,2,50))
print('Constant policy with 2 seats p=100 \t:',simulateConstant(customers,2,100))

Constant policy with 2 seats p=50 	: 100
Constant policy with 2 seats p=100 	: 100


In [12]:
def simulateDynamic(customers, seats, p1, p2):   
    countTicket = seats
    revenue = 0
    for x in range(len(customers)):
        if customers.loc[x, 'arrival_time'] > 14:
            
            if customers.loc[x, 'valuation'] >= p1:
                countTicket = countTicket - 1
                revenue = revenue + p1
            if countTicket == 0:
                break
        else:
            if customers.loc[x, 'valuation'] >= p2:
                countTicket = countTicket - 1
                revenue = revenue + p2
            if countTicket == 0:
                break
    
    return revenue

In [13]:
print('Dynamic policy with 3 seats p1=40, p2=95 \t:',simulateDynamic(customers,3,40,95))
print('Dynamic policy with 2 seats p1=40, p2=95 \t:',simulateDynamic(customers,2,40,95))
print('Dynamic policy with 1 seats p1=50, p2=100 \t:',simulateDynamic(customers,1,50,100))

Dynamic policy with 3 seats p1=40, p2=95 	: 175
Dynamic policy with 2 seats p1=40, p2=95 	: 80
Dynamic policy with 1 seats p1=50, p2=100 	: 50


In [15]:
def simulateRationing(customers, seats, p1, p2,l):   
    full = customers
    countTicket = seats
    revenue = 0
    p = p1
    for x in range(len(full)):
        if countTicket <= l:
            p = p2
        if full.loc[x, 'valuation'] >= p:
            countTicket = countTicket - 1
            revenue = revenue + p
        if countTicket == 0:
            break
    return revenue

In [16]:
print('Rationing policy with 2 seats p1=40, p2=100, l=1 \t:',\
simulateRationing(customers,2,40,100,1))
print('Rationing policy with 1 seats p1=40, p2=95, l=1 \t:',\
simulateRationing(customers,1,40,95,1))

Rationing policy with 2 seats p1=40, p2=100, l=1 	: 140
Rationing policy with 1 seats p1=40, p2=95, l=1 	: 95


## 4. Optimizing Parameters

In [17]:
def lists():
    lists =[]
    for i in range(100):
        np.random.seed(i)
        lists.append(generateCustomers())
    return lists

In [18]:
customerlist = lists()

In [19]:
len(customerlist)

100

In [23]:
def optimizerConstant():
    data = []
    total = 0
    for price in range(200,300,5):
        for customers in customerlist:
            total+=simulateConstant(customers, 100, price)
        profit = total/100 - 20000
        data.append([price, profit])
        total = 0
    data = pd.DataFrame(data, columns = ['Price', 'Profit'])
    maxindex = data['Profit'].idxmax()
    optprice = data.iloc[maxindex,:]
    return optprice

In [24]:
opt1 = optimizerConstant()
profit1 = opt1.loc['Profit'] 
price1 = opt1.loc['Price']
print("Constant price policy: best p = ", price1, " , best expected profit = ", profit1)

Constant price policy: best p =  245.0  , best expected profit =  2633.0999999999985


In [27]:
def optimizerDynamic():
    data = []
    total = 0
    for price1 in range(200,300,5):
        for price2 in range(240,400,5):
            for customers in customerlist: 
                total+=simulateDynamic(customers, 100, price1, price2)
            profit = total/100 - 20000
            data.append([price1,price2,profit])
            total = 0
    data = pd.DataFrame(data, columns = ['Price1','Price2','Profit'])
    maxindex = data['Profit'].idxmax()
    optprice = data.iloc[maxindex,:]
    return optprice

In [28]:
opt2 = optimizerDynamic()
profit2 = opt2.loc['Profit'] 
price21 = opt2.loc['Price1']
price22 = opt2.loc['Price2']
print("Dynamic price policy: best p1 = ", price21, " , best p2 = ", price22, ", best expected profit = ", profit2)

Dynamic price policy: best p1 =  240.0  , best p2 =  300.0 , best expected profit =  3506.7999999999993


In [30]:
def optimizerCapacity():
    data = []
    total = 0
    for price1 in range(200,300,5):
        for price2 in range(240,400,5):
            for l in range(10,40,1):
                for customers in customerlist:
                    total+=simulateRationing(customers, 100, price1, price2,l)
                profit = total/100 - 20000
                data.append([price1,price2,l,profit])
                total = 0
    data = pd.DataFrame(data, columns = ['Price1','Price2','Left','Profit'])
    maxindex = data['Profit'].idxmax()
    optprice = data.iloc[maxindex,:]
    return optprice

In [ ]:
opt3 = optimizerCapacity()
profit3 = opt3.loc['Profit'] 
price31 = opt3.loc['Price1']
price32 = opt3.loc['Price2']
l = opt3.loc['Left']
print("Capacity rationing price policy: best p1 = ", price31, " , best p2 = ", price32, ", best l = ",l,", best expected profit = ", profit3)

In [ ]:
Capacity rationing price policy: best p1 =  230.0  , best p2 =  335.0 , best l =  28.0 , best expected profit =  3559.75